In [1]:
import os

In [2]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project'

In [5]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/Food_Time_Data_Set.csv")
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,temperature,humidity,precipitation,weather_description,Traffic_Level,Distance (km),TARGET
0,B379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,19.50,93.0,0.0,mist,Very High,37.17,85.266667
1,5D6D,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,20.45,91.0,0.0,mist,Low,3.34,28.583333
2,7A6A,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,23.86,78.0,0.0,mist,Moderate,10.05,35.183333
3,70A2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,26.55,87.0,0.0,mist,High,9.89,43.450000
4,9BB4,HYDRES09DEL03,22.0,4.8,17.431668,78.408321,17.461668,78.438321,Buffet,motorcycle,21.43,65.0,0.0,broken clouds,Moderate,11.30,30.600000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9035 entries, 0 to 9034
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           9035 non-null   object 
 1   Delivery_person_ID           9035 non-null   object 
 2   Delivery_person_Age          9035 non-null   float64
 3   Delivery_person_Ratings      9035 non-null   float64
 4   Restaurant_latitude          9035 non-null   float64
 5   Restaurant_longitude         9035 non-null   float64
 6   Delivery_location_latitude   9035 non-null   float64
 7   Delivery_location_longitude  9035 non-null   float64
 8   Type_of_order                9035 non-null   object 
 9   Type_of_vehicle              9035 non-null   object 
 10  temperature                  9035 non-null   float64
 11  humidity                     9035 non-null   float64
 12  precipitation                9035 non-null   float64
 13  weather_descriptio

In [7]:
data.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Type_of_order                  0
Type_of_vehicle                0
temperature                    0
humidity                       0
precipitation                  0
weather_description            0
Traffic_Level                  0
Distance (km)                  0
TARGET                         0
dtype: int64

In [8]:

data.shape

(9035, 17)

In [9]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_dir: Path
    all_schema: dict

In [10]:
from src.datascience.constant import *
from src.datascience.utils.comman import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config["data_validation"]
        schema = self.schema["COLUMNS"]

        create_directories([config["root_dir"]])

        data_validation_config = DataValidationConfig(
            root_dir=config["root_dir"],
            STATUS_FILE=config["STATUS_FILE"],
            unzip_dir =config["unzip_dir"],
            all_schema=schema
        )
        return data_validation_config


In [12]:
import os
from src.datascience import logger

In [13]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True  # Default to True

            data = pd.read_csv(self.config.unzip_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False # No need to continue checking once invalid column found

            with open(self.config.STATUS_FILE, "w") as file:
                file.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e


In [18]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-02-12 19:35:19,181: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-02-12 19:35:19,184: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-02-12 19:35:19,203: INFO: comman: yaml file: schema.yaml loaded successfully]
[2025-02-12 19:35:19,206: INFO: comman: created directory at: artifacts]
[2025-02-12 19:35:19,208: INFO: comman: created directory at: artifacts/data_validation]
